In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc
plt.style.use('fivethirtyeight')
import sqlite3

conn = sqlite3.connect('/Users/adrian/Desktop/NBA_Machine_Learning/data/dataset.sqlite')

# Use double quotes around the table name
df = pd.read_sql_query('SELECT * FROM "dataset_2012-24_new"', conn)

# close the connection
conn.close()

# drop the index column
df.drop('index', axis=1, inplace=True)
# change 'Home-Team-Win' to 'Target'
df = df.rename(columns={'Home-Team-Win': 'Target'})


# save orginal data
df_original = df.copy()
df_original.to_parquet("/Users/adrian/Desktop/NBA_Machine_Learning/data/original_df.parquet")



# Data Cleaning

In [10]:
# find all time gaps and plot
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')
df['Time_Gap'] = df['Date'].diff().dt.days

df['Season'] = 0
season = 2012
# Iterate through the DataFrame to set the season based on time gaps
for i in range(len(df)):
    if df['Time_Gap'].iloc[i] > 80:
        df.at[i, 'Season'] = season
        season += 1
    else:
        df.at[i, 'Season'] = season

df = df.drop(columns=['Time_Gap'])
print(len(df))

# rename Days-Rest-Home and Days-Rest-Away to Days-Rest and Days-Rest.1
df = df.rename(columns={'Days-Rest-Home': 'Days-Rest', 'Days-Rest-Away': 'Days-Rest.1'})



15115


# Feature Engineering

In [11]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def calculate_elo_chronologically(data, initial_elo=1500, k=20, home_advantage=100):
    """Calculate Elo ratings chronologically without looking ahead"""
    data = data.sort_values('Date').copy()
    
    # Initialize Elo tracking
    team_elos = {}
    season_end_elos = {}
    elo_history = []
    
    for idx, row in tqdm(data.iterrows(), total=len(data), desc="Calculating Elo ratings"):
        home_team = row['TEAM_NAME']
        away_team = row['TEAM_NAME.1']
        season = row['Season']
        
        # Initialize teams if not seen before
        if home_team not in team_elos:
            # Check if we have previous season data and it's not the first season
            if season > min(data['Season']) and season in season_end_elos and home_team in season_end_elos[season-1]:
                # Use 75% of last season's ending Elo + 25% of initial Elo
                team_elos[home_team] = 0.75 * season_end_elos[season-1][home_team] + 0.25 * initial_elo
            else:
                team_elos[home_team] = initial_elo
                
        if away_team not in team_elos:
            # Check if we have previous season data and it's not the first season
            if season > min(data['Season']) and season in season_end_elos and away_team in season_end_elos[season-1]:
                team_elos[away_team] = 0.75 * season_end_elos[season-1][away_team] + 0.25 * initial_elo
            else:
                team_elos[away_team] = initial_elo
        
        # Store pre-game Elos
        home_elo = team_elos[home_team]
        away_elo = team_elos[away_team]
        elo_history.append((home_elo, away_elo))
        
        # Update Elos based on game result
        home_win = row['Target']
        
        # Calculate expected scores
        elo_diff = home_elo - away_elo + home_advantage
        home_expected = 1 / (1 + 10 ** (-elo_diff / 400))
        
        # Update Elo ratings
        home_new = home_elo + k * (home_win - home_expected)
        away_new = away_elo + k * ((1 - home_win) - (1 - home_expected))
        
        team_elos[home_team] = home_new
        team_elos[away_team] = away_new
        
        # Store season-end Elos
        if season not in season_end_elos:
            season_end_elos[season] = {}
        season_end_elos[season][home_team] = home_new
        season_end_elos[season][away_team] = away_new
    
    # Add Elo columns to dataframe
    data['Elo_Team'] = [x[0] for x in elo_history]
    data['Elo_Team.1'] = [x[1] for x in elo_history]
    
    return data

# Rest of the code remains the same
def calculate_historical_stats(data):
    """Calculate historical win percentages using only past games"""
    data = data.sort_values('Date').copy()
    
    # Initialize tracking dictionaries
    team_stats = {}
    season_stats = {}
    
    # Columns to store results
    data['home_win_pct'] = 0.0
    data['away_win_pct'] = 0.0
    data['total_games'] = 0
    
    for idx, row in tqdm(data.iterrows(), total=len(data), desc="Calculating historical stats"):
        home_team = row['TEAM_NAME']
        away_team = row['TEAM_NAME.1']
        season = row['Season']
        
        # Initialize season stats if needed
        if season not in season_stats:
            season_stats[season] = {}
            
        # Initialize team stats if needed
        for team in [home_team, away_team]:
            if team not in season_stats[season]:
                season_stats[season][team] = {'wins': 0, 'losses': 0, 'games': 0}
        
        # Get current stats before updating
        home_stats = season_stats[season][home_team]
        away_stats = season_stats[season][away_team]
        
        # Store historical stats
        data.at[idx, 'home_win_pct'] = home_stats['wins'] / max(home_stats['games'], 1)
        data.at[idx, 'away_win_pct'] = away_stats['wins'] / max(away_stats['games'], 1)
        data.at[idx, 'total_games'] = home_stats['games']
        
        # Update stats after game
        game_result = row['Target']
        if game_result == 1:  # Home team won
            season_stats[season][home_team]['wins'] += 1
            season_stats[season][away_team]['losses'] += 1
        else:  # Away team won
            season_stats[season][home_team]['losses'] += 1
            season_stats[season][away_team]['wins'] += 1
            
        season_stats[season][home_team]['games'] += 1
        season_stats[season][away_team]['games'] += 1
    
    return data

def process_data_without_leakage(data):
    """Process the entire dataset without data leakage"""
    # Sort data chronologically
    data = data.sort_values('Date').copy()
    
    # Calculate Elo ratings
    data = calculate_elo_chronologically(data)
    
    # Calculate historical statistics
    data = calculate_historical_stats(data)
    
    return data

data_processed = process_data_without_leakage(df)

Calculating Elo ratings:   0%|          | 0/15115 [00:00<?, ?it/s]

Calculating historical stats: 100%|██████████| 15115/15115 [00:00<00:00, 17880.47it/s]


In [13]:




# save data
df.to_parquet("/Users/adrian/Desktop/NBA_Machine_Learning/data/processed_dataset.parquet", index=False)